In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pandas as pd

/tmp/ipykernel_464529/362687639.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#save_dir = '/home/jl277509/Documents/Ablation_2_models/translation.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/7_translation'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimdepth_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/8_trimdepth_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimextremities_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/9_trimextremities_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/cutin_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/10_cutin_translation_3'

save_dir = '/home/jl277509/Documents/Ablation_2_models/cutout_translation_3.csv'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'

save_dir = '/home/jl277509/Documents/Ablation_2_models/6_no_augm.csv'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'
n_models = 2

save_dir = '/home/jl277509/Documents/Ablation_2_models/rotation_translation.csv'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models/rotation_translation'
n_models = 1

# Step 1 : recover all perf values (with cross val std)

In [3]:
sub_dirs = {'SOr': [f'{basedir}/SOr_left_UKB40/', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'cv_score', 'cv_std', 'splits_score'], # NB : error in pipeline stds -> std ...
            'FIP': [f'{basedir}/FIP_right_UKB40/', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'cv_score', 'cv_std', 'splits_score'],
            'SC-sylv': [f'{basedir}/SC-sylv_left_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_central_left_dim{k}/test_values.json' for k in range(1,7)], 'cv_score', 'cv_std', 'splits_score'],
            'Cing_1': [f'{basedir}/LARGE_CINGULATE_right_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_cingulate_right_dim{k}/test_values.json' for k in range(1,7)], 'cv_score', 'cv_std', 'splits_score'],
            'Cing_2': [f'{basedir}/LARGE_CINGULATE_right_UKB40/', ['ACC_custom_embeddings/Right_PCS/test_values.json'], 'cv_score', 'cv_std', 'splits_score']
            }

l = os.listdir(basedir)
root_names = [elem for elem in l if ('UKB40' not in elem) and '#' not in elem]

In [4]:
# store everything globally
l_model_glob = []
l_perf_glob = []
l_std_glob = []
l_splits_scores_glob = []
l_region_glob = []
l_task_glob = []


for region, (sub_dir, auc_files, score, std, split_score) in sub_dirs.items(): 
    for auc_file in auc_files:
        models = os.listdir(sub_dir)
        models = [elem for elem in models if 'config' not in elem and 'multirun' not in elem]
        models.sort()
        task = Path(auc_file).parts[-2]
        for name in root_names:
            #l_models = [elem for elem in models if elem.startswith(name)]
            l_models = [elem for elem in models if name in elem]
            l_perfs = []
            l_stds = []
            l_splits_scores = []
            for model in l_models:
                try:
                    with open(os.path.join(sub_dir, model, auc_file), 'r') as f:
                        perf = json.load(f)
                except Exception:
                    print(f'{os.path.join(sub_dir, model, auc_file)} not found') ## NB : can't continue if not all files found ...

                if isinstance(perf[score], float):
                    value = perf[score]
                else:
                    value = perf[score][0]
                l_perfs.append(value)
                if isinstance(perf[std], float):
                    value = perf[std]
                else:
                    value = perf[std][0]
                l_stds.append(value)
                value = perf[split_score] # is a list
                l_splits_scores.append(value)
                
            l_model_glob.append(l_models)
            l_perf_glob.append(l_perfs)
            l_std_glob.append(l_stds)
            l_splits_scores_glob.append(l_splits_scores)
            l_region_glob.append([region for _ in range(len(l_perfs))])
            l_task_glob.append([task for _ in range(len(l_perfs))])

In [5]:
# reformat lists and put inside DataFrame
def flatten_sum(matrix):
    return sum(matrix, [])
cols = ['model', 'region', 'task', 'score', 'std']
values = np.array([flatten_sum(l_model_glob), flatten_sum(l_region_glob), flatten_sum(l_task_glob), flatten_sum(l_perf_glob), flatten_sum(l_std_glob)]).T
df = pd.DataFrame(data=values, columns=cols)
df['splits_score']=flatten_sum(l_splits_scores_glob)

# Step 2 : reorganize the df with the parameters

Rotation - Translation

In [6]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude_trans=[]
list_magnitude_rot=[]
for model in df['model'].tolist():
    if 'trans_0' in model:
        list_magnitude_trans.append(0)
    elif 'trans_3' in model:
        list_magnitude_trans.append(3)
    elif 'trans_6' in model:
        list_magnitude_trans.append(6)

    if 'rot_6' in model:
        list_magnitude_rot.append(6)
    elif 'rot_12' in model:
        list_magnitude_rot.append(12)


df['Rotation'] = list_magnitude_rot
df['Translation']=list_magnitude_trans
parameters=['Rotation', 'Translation']

Translation

In [ ]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'translation_1' in model:
        list_magnitude.append(1)
    elif 'translation_2' in model:
        list_magnitude.append(2)
    elif 'translation_3' in model:
        list_magnitude.append(3)
    elif 'translation_4' in model:
        list_magnitude.append(4)
    elif 'translation_5' in model:
        list_magnitude.append(5)
    elif 'translation_6' in model:
        list_magnitude.append(6)

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)
    elif 'p100' in model:
        list_proba.append(100) 

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

Trimdepth

In [ ]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_mode=[]
list_proba=[]
for model in df['model'].tolist():
    if '2mm' in model:
        list_magnitude.append(2)
    elif '4mm' in model:
        list_magnitude.append(4)
    elif '6mm' in model:
        list_magnitude.append(6)
    elif '8mm' in model:
        list_magnitude.append(8)

    if 'constant' in model:
        list_mode.append('constant')
    else:
        list_mode.append('per branch')

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Magnitude']=list_magnitude
df['Mode']=list_mode
df['Proba'] = list_proba
parameters=['Magnitude', 'Mode', 'Proba']

Trimextremities

In [31]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'pepper50' in model:
        list_magnitude.append(50)
    elif 'pepper100' in model:
        list_magnitude.append(100)

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

cutin

In [75]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '20_40' in model:
        list_magnitude.append('[20,40]')
    elif '30_50' in model:
        list_magnitude.append('[30,50]')
    elif '40_60' in model:
        list_magnitude.append('[40,60]')
    elif '50_70' in model:
        list_magnitude.append('[50,70]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

cutout

In [88]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '5_15' in model:
        list_magnitude.append('[5,15]')
    elif '10_20' in model:
        list_magnitude.append('[10,20]')
    elif '15_25' in model:
        list_magnitude.append('[15,25]')
    elif '20_30' in model:
        list_magnitude.append('[20,30]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

In [7]:
df_reformat = df.copy()
df_reformat = df_reformat.drop_duplicates(['model', 'task']).reset_index(drop=True)
df_reformat = df_reformat.drop(labels='model', axis='columns')
df_reformat['training_number']=[k%n_models for k in range(len(df_reformat))]
parameters.append('training_number')

In [8]:
# Pivoting the table
df_pivot = df_reformat.pivot(index=parameters, columns=['task', 'region'], values=['score', 'std', 'splits_score'])
df_pivot = df_pivot.dropna()

# Renaming columns
df_pivot.columns = [f'score_{col}' for col in df_pivot.columns]

# Resetting index to get a flat DataFrame
df_pivot = df_pivot.reset_index()

isomap_score_cols = [elem for elem in df_pivot.columns if ("score_('score', 'Isomap" in elem) and ("SC-sylv" in elem)]
df_pivot['score_Isomap_SC-sylv_left_global'] = df_pivot[isomap_score_cols].astype(float).mean(axis=1)

isomap_score_cols = [elem for elem in df_pivot.columns if ("score_('score', 'Isomap" in elem) and ("Cing" in elem)]
df_pivot['score_Isomap_Cing_right_global'] = df_pivot[isomap_score_cols].astype(float).mean(axis=1)

In [12]:
df_pivot

,Rotation,Translation,training_number,"score_('score', 'Left_OFC', 'SOr')","score_('score', 'Right_FIP', 'FIP')","score_('score', 'Isomap_central_left_dim1', 'SC-sylv')","score_('score', 'Isomap_central_left_dim2', 'SC-sylv')","score_('score', 'Isomap_central_left_dim3', 'SC-sylv')","score_('score', 'Isomap_central_left_dim4', 'SC-sylv')","score_('score', 'Isomap_central_left_dim5', 'SC-sylv')",...,"score_('splits_score', 'Isomap_central_left_dim6', 'SC-sylv')","score_('splits_score', 'Isomap_cingulate_right_dim1', 'Cing_1')","score_('splits_score', 'Isomap_cingulate_right_dim2', 'Cing_1')","score_('splits_score', 'Isomap_cingulate_right_dim3', 'Cing_1')","score_('splits_score', 'Isomap_cingulate_right_dim4', 'Cing_1')","score_('splits_score', 'Isomap_cingulate_right_dim5', 'Cing_1')","score_('splits_score', 'Isomap_cingulate_right_dim6', 'Cing_1')","score_('splits_score', 'Right_PCS', 'Cing_2')",score_Isomap_SC-sylv_left_global,score_Isomap_Cing_right_global
0,6,0,0,0.6563381068664809,0.7183887837988205,0.3125370413700635,0.23454209828913308,0.335207060028459,0.2525250251497162,0.036965615204267,...,"[0.0028481591649851534, 0.030726880077168284, ...","[0.05196097733772587, 0.02381836097901957, 0.0...","[0.15101852406300265, 0.07495121045976438, -0....","[0.06184827353832856, 0.12615577061143102, 0.0...","[0.14306253268117686, 0.1024982229711614, 0.19...","[0.01764778978599013, -0.04254423846501143, 0....","[-0.009224282659214156, -0.01440500152393187, ...","[0.7701612903225806, 0.7585227272727272, 0.543...",0.197420,0.063843
1,6,3,0,0.6165734500511184,0.8069433626349172,0.4833713280567494,0.3025195407869884,0.3574749040398618,0.1934935725457313,0.027582431052956012,...,"[0.07762243205450103, 0.12343094995819126, 0.1...","[0.09082426841115643, 0.05781129252771611, 0.0...","[0.2600696149993431, 0.24699870563517157, 0.04...","[0.08160643292390513, 0.16648011987343714, 0.0...","[0.19440675663305484, 0.11779665629313896, 0.1...","[0.001529448413182255, -0.01787379862407734, 0...","[-0.011426420671142079, -0.014731532789485247,...","[0.7419354838709677, 0.8224431818181818, 0.719...",0.241067,0.088011
2,6,6,0,0.6918058147775948,0.8061377545343273,0.4878465942259914,0.38019975316157967,0.4830662290335395,0.21411171511343308,0.007916400534376278,...,"[0.10340387637137893, 0.10378725825196189, 0.1...","[0.049255799140188516, 0.07523981803796831, 0....","[0.29227455605335206, 0.2209117043264366, 0.30...","[0.08268836350902153, 0.16606164037092108, 0.1...","[0.2232574668639955, 0.16429286832463652, 0.20...","[-0.006747783598417767, -0.006647652853311259,...","[0.019193019433111913, -0.04581949252277351, -...","[0.7930107526881721, 0.8181818181818181, 0.808...",0.274570,0.111406
3,12,0,0,0.6942360339252058,0.8298308668076109,0.5188987821015878,0.43441895727699836,0.6116576554751274,0.40591794864006603,0.15788789534788572,...,"[-0.02369695189884724, 0.07431922327678786, 0....","[0.085089414310484, 0.06363039177772378, 0.042...","[0.2758252629728588, 0.30542669217455887, 0.16...","[0.12339727683131352, 0.13949379387704697, 0.1...","[0.2058850518857035, 0.1087235791695711, 0.229...","[0.012991120142501256, -0.023364310388992227, ...","[0.04783386963588743, -0.05707542566219548, -0...","[0.7567204301075269, 0.8153409090909092, 0.633...",0.362659,0.115234
4,12,3,0,0.6064622152177966,0.8292500278179593,0.5675807159885797,0.36415602126353885,0.5956463333404705,0.3055674322152483,0.13374310840608508,...,"[0.09926488107434739, 0.1456303781825875, 0.14...","[0.11202232556432279, 0.05996502554779248, 0.0...","[0.32371529823896195, 0.324034944680339, 0.303...","[0.04701396976701455, 0.12434165431556643, 0.0...","[0.18820484708082807, 0.15878626829619036, 0.1...","[0.01715733734409497, 0.007731866229804818, 0....","[-0.009313816226599547, -0.015009554508850886,...","[0.6881720430107527, 0.9105113636363636, 0.779...",0.343618,0.100719
5,12,6,0,0.6103800018212636,0.8090875709357963,0.49208007984949764,0.32169010106897533,0.43195263532647815,0.1264291113

In [17]:
df_scores = df_pivot[["score_('score', 'Left_OFC', 'SOr')", "score_('score', 'Right_FIP', 'FIP')", "score_('score', 'Right_PCS', 'Cing_2')",
                      "score_Isomap_SC-sylv_left_global", "score_Isomap_Cing_right_global"]].apply(pd.to_numeric).mul(100).round(1)
df_scores.columns=['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left', 'Isomap_Cing_right']

# get stds for Isomap
# SC-sylv
df_splits_score_isomap = df_pivot[[f"score_('splits_score', 'Isomap_central_left_dim{k}', 'SC-sylv')" for k in range(1,7)]]
avg_per_split = df_splits_score_isomap.apply(lambda row: np.mean(row.tolist(), axis=0), axis=1)
std_isomap_sc_sylv = avg_per_split.apply(lambda row : np.std(row.tolist()))
# Cing
df_splits_score_isomap = df_pivot[[f"score_('splits_score', 'Isomap_cingulate_right_dim{k}', 'Cing_1')" for k in range(1,7)]]
avg_per_split = df_splits_score_isomap.apply(lambda row: np.mean(row.tolist(), axis=0), axis=1)
std_isomap_cing = avg_per_split.apply(lambda row : np.std(row.tolist()))

# get the std df
df_stds = df_pivot[["score_('std', 'Left_OFC', 'SOr')", "score_('std', 'Right_FIP', 'FIP')", "score_('std', 'Right_PCS', 'Cing_2')"]]
df_stds['isomap_SC-sylv_left']=std_isomap_sc_sylv
df_stds['isomap_Cing_right']=std_isomap_cing
df_stds = df_stds.apply(pd.to_numeric).mul(100).round(1)
df_stds.columns=['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left', 'Isomap_Cing_right']

# combine in one df
formatted_df = df_scores.astype(str) + "(" + df_stds.astype(str) + ")"

formatted_df = pd.concat((df_pivot[parameters], formatted_df), axis=1)

/tmp/ipykernel_464529/1976764873.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stds['isomap_SC-sylv_left']=std_isomap_sc_sylv
/tmp/ipykernel_464529/1976764873.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stds['isomap_Cing_right']=std_isomap_cing


In [18]:
formatted_df

,Rotation,Translation,training_number,SOr,FIP,PCS,Isomap_SC-sylv_left,Isomap_Cing_right
0,6,0,0,65.6(4.2),71.8(8.5),67.3(8.4),19.7(3.5),6.4(2.2)
1,6,3,0,61.7(3.4),80.7(5.0),75.6(4.1),24.1(2.2),8.8(1.7)
2,6,6,0,69.2(1.6),80.6(6.2),81.4(2.3),27.5(2.7),11.1(1.0)
3,12,0,0,69.4(1.7),83.0(3.8),72.9(6.8),36.3(4.0),11.5(2.3)
4,12,3,0,60.6(2.7),82.9(5.2),80.8(9.0),34.4(2.1),10.1(0.9)
5,12,6,0,61.0(3.6),80.9(7.3),89.4(5.0),24.0(2.7),12.5(3.1)


# Save df

In [93]:
formatted_df.to_csv(save_dir, index=False)